In [44]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD_Python_Module import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "password"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'], inplace=True, errors='ignore')


## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard'))),

    # Logo and unique identifier
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                         style={'height': '80px'})),
    html.Center(html.Div("Created by Ashley Sapunarich - October 2025",
                         style={'fontSize': 16, 'marginBottom': '20px'})),

    html.Hr(),

    # Interactive filtering options
    html.Div([
        html.Label("Select Rescue Type:"),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset',
            labelStyle={'display': 'block'}
        )
    ], style={'marginLeft': '20px'}),

    html.Hr(),

    # Interactive DataTable
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        sort_action='native',
        filter_action='native',
        row_selectable='multi',
        selected_rows=[],
        style_table={'overflowX': 'auto'},
        style_cell={
            'minWidth': '100px', 'width': '150px', 'maxWidth': '200px',
            'whiteSpace': 'normal',
            'textAlign': 'left'
        },
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        },
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)'
            }
        ]
    ),

    html.Br(),
    html.Hr(),

    # Side-by-side charts
    html.Div(className='row',
         style={'display': 'flex', 'flexWrap': 'wrap', 'gap': '20px'},
         children=[
             html.Div(id='graph-id', style={
                 'flex': '1',
                 'minWidth': '300px',
                 'padding': '10px'
             }),
             html.Div(id='map-id', style={
                 'flex': '1',
                 'minWidth': '300px',
                 'padding': '10px'
             })
         ])
])


#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    # Define rescue filters
    rescue_filters = {
        'Water Rescue': {'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']}},
        'Mountain or Wilderness Rescue': {'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Siberian Husky', 'Rottweiler']}},
        'Disaster or Individual Tracking': {'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound']}},
        'Reset': {}  # No filter
    }

    # Get query based on selection
    query = rescue_filters.get(filter_type, {})
    results = db.read(query)

    # Convert to DataFrame and return as dict
    df = pd.DataFrame.from_records(results)
    return df.to_dict('records')

@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data')]
)
def update_graph(viewData):
    if not viewData:
        return [html.Div("No data available to display chart.")]

    dff = pd.DataFrame(viewData)

    if 'breed' not in dff.columns or dff.empty:
        return [html.Div("No breed data available.")]

    breed_counts = dff['breed'].value_counts().reset_index()
    breed_counts.columns = ['breed', 'count']

    fig = px.pie(breed_counts, names='breed', values='count', title='Preferred Breeds by Quantity')

    return [dcc.Graph(figure=fig)]

#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:
        return []

    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, selected_rows):
    if not viewData:
        return [html.Div("No location data available.")]

    dff = pd.DataFrame(viewData)

    # Check for coordinate columns
    if 'location_lat' not in dff.columns or 'location_long' not in dff.columns:
        return [html.Div("Missing location coordinates.")]

    dff = dff.dropna(subset=['location_lat', 'location_long'])

    if dff.empty:
        return [html.Div("No valid coordinates to display.")]

    # If a row is selected, show a Leaflet marker
    if selected_rows:
        row = selected_rows[0]
        if row < len(dff):
            lat = dff.iloc[row]['location_lat']
            lon = dff.iloc[row]['location_long']
            breed = dff.iloc[row]['breed']
            name = dff.iloc[row]['name']

            return [
                dl.Map(style={'width': '100%', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
                    dl.TileLayer(id="base-layer-id"),
                    dl.Marker(position=[lat, lon], children=[
                        dl.Tooltip(breed),
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(name)
                        ])
                    ])
                ])
            ]

    # If no row is selected, show all points on a Plotly map
    fig = px.scatter_mapbox(
        dff,
        lat='location_lat',
        lon='location_long',
        hover_name='name',
        zoom=10,
        height=400
    )
    fig.update_layout(mapbox_style="open-street-map")

    return [dcc.Graph(figure=fig)]



app.run_server(debug=True)


Dash app running on https://songmemphis-lobbyviva-3000.codio.io/proxy/8050/
